# WebScraping Project UpWork - Dec.2023

## Best USA Lawyers

In [1]:
%%capture
!pip install --upgrade pip
!pip install --upgrade selenium
#!conda install -c conda-forge webdriver-manager
!pip install osascript
!pip install requests
!pip install bs4
!pip install urllib3
!pip install openpyxl

In [7]:
import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import json

import os
import time
import osascript

from urllib.parse import unquote



In [52]:
#url='https://www.bestlawyers.com/search/best-lawyers/us'
url = 'https://www.bestlawyers.com/search?pub-list-id=1&cc=US&page=1&sort=relevancy&lawyers=1'


In [53]:
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)
driver.maximize_window()

In [55]:
def extract_location_from_google_maps_link(google_maps_link):

    decoded_link = unquote(google_maps_link)

    start_index = decoded_link.find("/search/") + len("/search/")
    location_text = decoded_link[start_index:]
    
    location_text = location_text.replace("+", " ").replace("%2c", ",")

    return location_text

In [76]:
lawyers_df = pd.DataFrame(columns=['Name', 'Website', 'Address', 'Phone', 'Firm'])
data = {
    'names' : [],
    'websites' : [],
    'addresses' : [],
    'phones' : [],
    'firms' : []
}


for i in range(1,21):
    next_url = f'https://www.bestlawyers.com/search?pub-list-id=1&cc=US&page={i}&sort=relevancy&lawyers=1'
    driver.get(next_url)

    #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="grid_ce05a831aaef47c29a0f6563b79c94f0"]')))
    time.sleep(2)
    
    html  = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    cards = soup.find_all('div', {'class':['card-scaler']})

    for k, card in enumerate(cards):

        
        lawyer_url = 'https://www.bestlawyers.com/'+card.find('a')['href']
        
        #### LAWYER URL ####
        driver.get(lawyer_url)
        time.sleep(1)
        
        html_lawyer = driver.page_source
        soup_lawyer = BeautifulSoup(html_lawyer, 'html.parser')
        #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/main/div[1]/div[1]/div[1]/div[1]/h1/a")))
        
        
        #### NAME ####
        try:
            name = soup_lawyer.find('h1', class_='lawyer-name').text.strip()
            #name = driver.find_element(By.XPATH, '/html/body/main/div[1]/div[1]/div[1]/div[1]/h1/a').text
            data['names'].append(name)
        except:
            print(f'Error al buscar name para {name}. Index {k}')
            data['name'].append('')


        #### WEBSITE ####
        try:
            website = soup_lawyer.find('a', class_ = 'ga-lawyer-link')['href']
            data['websites'].append(website)
        except:
            print(f'Error al buscar website para {name}. Index {k}')
            data['websites'].append('')

        
        #### ADDRESS ####
        try:
            gmap_link = soup_lawyer.find('a', {'title': 'See map location'})['href']
            address = extract_location_from_google_maps_link(gmap_link)
            data['addresses'].append(address)
        except:
            print(f'Error al buscar address para {name}. Index {k}')
            data['addresses'].append('')

        
        #### PHONE ####
        try:
            phone = soup_lawyer.find('a', class_ = 'ga-lawyer-phone')['href'][4:]
            data['phones'].append(phone)
        except:
            print(f'Error al buscar phone para {name}. Index {k}')
            data['phones'].append('')


        #### FIRM ####
        try:
            firm = driver.find_element(By.XPATH, '/html/body/main/div[1]/div[1]/div[1]/div[1]/p[1]/a').text
            data['firms'].append(firm)
        except:
            print(f'Error al buscar firm para {name}. Index {k}')
            data['firms'].append('')
        
        
        
        print(f'Done with {name}!')
        driver.back()
        #print(data)
        time.sleep(1)

    print('\n=====Sleeping before next lawyers...=====\n')    
    time.sleep(2)



actions = ActionChains(driver)
actions.move_to_element(driver.find_element(By.XPATH, '//*[@id="grid_6b709172227b4fd0ae04aee78cb86ff9"]/div[4]/div/button[1]')).perform()
        

Error al buscar phone para Tara Aaron-Stelluto. Index 0
Done with Tara Aaron-Stelluto!
Done with William D. "Bill" Aaron, Jr.!
Error al buscar phone para Mark J. Abate. Index 2
Done with Mark J. Abate!
Done with Donald R. "Don" Abaunza!
Error al buscar phone para Carolyn Abbate. Index 4
Done with Carolyn Abbate!
Done with Allison Abbott!
Error al buscar phone para Charles H. Abbott. Index 6
Done with Charles H. Abbott!
Error al buscar phone para Hirschel T. Abbott, Jr.. Index 7
Done with Hirschel T. Abbott, Jr.!
Done with Scott M. Abbott!
Error al buscar phone para Susan L. "Sue" Abbott. Index 9
Done with Susan L. "Sue" Abbott!
Error al buscar phone para Michael V. "Mike" Abcarian. Index 10
Done with Michael V. "Mike" Abcarian!
Error al buscar phone para Mary Clift Abdalla. Index 11
Done with Mary Clift Abdalla!
Error al buscar phone para Amy Abdo. Index 12
Done with Amy Abdo!
Error al buscar phone para George E. Abdo III. Index 13
Done with George E. Abdo III!
Error al buscar phone pa

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="grid_6b709172227b4fd0ae04aee78cb86ff9"]/div[4]/div/button[1]"}
  (Session info: chrome=119.0.6045.199); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000102dcdd28 chromedriver + 4795688
1   chromedriver                        0x0000000102dc52b3 chromedriver + 4760243
2   chromedriver                        0x000000010299e88d chromedriver + 407693
3   chromedriver                        0x00000001029ed3d9 chromedriver + 730073
4   chromedriver                        0x00000001029ed591 chromedriver + 730513
5   chromedriver                        0x0000000102a337e4 chromedriver + 1017828
6   chromedriver                        0x0000000102a14ccd chromedriver + 892109
7   chromedriver                        0x0000000102a3

In [79]:
lawyers_df['Name'] = data['names']
lawyers_df['Website'] = data['websites']
lawyers_df['Address'] = data['addresses']
lawyers_df['Phone'] = data['phones']
lawyers_df['Firm'] = data['firms']

In [91]:
lawyers_df.to_excel('best_lawyers.xlsx', index=False)

In [2]:
!pip install pipreqs

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=79cecfb723dc20ecc55917803a846180beca9257da56602fc9adb81d11a66f92
  Stored in directory: /Users/bpeco/Library/Caches/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
!pip3 freeze

alembic==1.13.0
appnope==0.1.3
asttokens==2.4.1
attrs==23.1.0
beautifulsoup4==4.12.2
bs4==0.0.1
cachetools==5.3.2
catboost==1.2.2
certifi==2023.11.17
charset-normalizer==3.3.2
colorlog==6.8.0
comm==0.2.0
contourpy==1.2.0
cycler==0.12.1
db-dtypes==1.1.1
debugpy==1.8.0
decorator==5.1.1
docopt==0.6.2
et-xmlfile==1.1.0
exceptiongroup==1.2.0
executing==2.0.1
fonttools==4.45.1
google-api-core==2.15.0
google-api-python-client==2.110.0
google-auth==2.25.2
google-auth-httplib2==0.1.1
google-auth-oauthlib==1.1.0
google-cloud-bigquery==3.13.0
google-cloud-bigquery-storage==2.23.0
google-cloud-core==2.4.1
google-crc32c==1.5.0
google-resumable-media==2.6.0
googleapis-common-protos==1.62.0
graphviz==0.20.1
greenlet==3.0.2
grpcio==1.60.0
grpcio-status==1.60.0
h11==0.14.0
httplib2==0.22.0
idna==3.6
imbalanced-learn==0.11.0
imblearn==0.0
ipykernel==6.26.0
ipython==8.18.0
jedi==0.19.1
joblib==1.3.2
jupyter_client==8.6.0
jupyter_core==5.5.0
kiwisolver==1.4.5
lightgbm==4.1.0
Mako==1.3.0
MarkupSafe==2.1.3
